imports

In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

variables

folder pathing

In [3]:
export_path = Path('./export')
export_path.mkdir(parents=True, exist_ok=True)
# Define the base filename
base_filename = 'alb_data'
file_extension = '.csv'
counter = 1
while True:
    file_path = export_path / f'{base_filename}{counter}{file_extension}'
    if not file_path.exists():
        break
    counter += 1

adding local csvs files

In [4]:
ingredients_file_path = "ingredients.csv"
recipes_file_path = "recipes.csv"
focus_file_path = "focus.csv"
ingredients_df = pd.read_csv(ingredients_file_path)
recipes_df = pd.read_csv(recipes_file_path)
focus_df = pd.read_csv(focus_file_path)

getting city names

In [5]:
suffixes_to_remove = [' Sell Order', ' Buy Order']
cities_set = set()
for column in ingredients_df.columns:
    # Check if the column ends with any of the specified suffixes
    for suffix in suffixes_to_remove:
        if column.endswith(suffix):
            # Remove the suffix to isolate the city name
            clean_name = column.replace(suffix, '').strip()
            cities_set.add(clean_name)
            break  # Break the loop once a valid suffix is found and processed


In [6]:
cities = sorted(list(cities_set))

In [7]:
print(cities)

['Bridgewatch', 'Caerleon', 'Fort Sterling', 'Lymhurst', 'Martlock', 'Thetford']


: 

In [172]:
recipes = recipes_df.columns.tolist()[1:]

c / 2^(x/10000) where c is the base focus cost and x is your focus specialization.

In [173]:
def single_item_achievable(cost, focus, focus_limit, budget_limit):
  """
  Analyzes if a single item can be purchased multiple times within focus and budget limit.

  Args:
      cost: Flat cost of the item.
      focus: Focus required per unit of the item.
      focus_limit: The total amount of focus available.
      budget_limit: The total budget available.

  Returns:
      A tuple containing the number of items achievable and a string indicating the limiting factor (if any).
  """
  # Check if focus requirement is within limit
  if focus > focus_limit:
    return 0, "Focus Limit Exceeded"  # Cannot achieve any item due to focus limitation
  elif (cost > budget_limit):
    return 0, "Cost Exceeded" # Cannot achieve any item due to budget limitation
  
  # Calculate maximum number of items affordable within budget
  affordable_items = budget_limit // cost

  # Limit by focus if affordable items exceed focus limit
  achievable_items = min(affordable_items, focus_limit // focus)

  # Determine limiting factor (if any)
  limiting_factor = ""
  if affordable_items > achievable_items:
    # Budget limited, could achieve more with additional focus
    additional_focus_needed = (affordable_items - achievable_items) * focus
    limiting_factor = f"Focus Limited (Could Afford {affordable_items} items)" \
                      f", Need {additional_focus_needed} more Focus."
  elif affordable_items < achievable_items:
    # Focus limited, could achieve more with additional budget
    additional_budget_needed = (achievable_items - affordable_items) * cost
    limiting_factor = f"Budget Limited (Could Achieve Upto {achievable_items:.2f} items)" \
                      f", Need ${additional_budget_needed:.2f} more Budget."

  return achievable_items, limiting_factor

In [174]:
def smallest_non_zero_round_down(series):
  """
  This function rounds down the elements in a pandas Series and then finds the
  smallest non-zero value. If all elements are zero, it returns 0.
  """
  rounded_series = series.floor()  # Round down all elements
  if rounded_series.sum() == 0:
    return 0  # All zeros, return 0
  else:
    return rounded_series.where(rounded_series != 0).min()

In [175]:
def craft_until_focus_depleted(achievable_items, focus, focus_limit, focus_percentage, merged_data):
    
    # Initialize variables
    total_crafts = achievable_items
    remaining_focus = focus_limit - (achievable_items * focus) # amount of focus left to use after inital
    remaining_ingredients = achievable_items * merged_data['Quantity']  # initial ingredients used

    while True:
        # Calculate ingredients returned
        ingredients_returned = (remaining_ingredients * (focus_percentage / 100))/ merged_data['Quantity'] # ingredients returned after using
        print("ingredients_returned\n",ingredients_returned)

        extra_crafts_based_on_ingredients = ingredients_returned.div(merged_data['Quantity']).fillna(0)
        print("part 1 extra_crafts_based_on_ingredients:\n", extra_crafts_based_on_ingredients)
        non_zero_rounded_down = np.floor(extra_crafts_based_on_ingredients[extra_crafts_based_on_ingredients> 0])
        # Finding the minimum of the rounded-down values or return 0 if empty
        min_value = non_zero_rounded_down.min() if not non_zero_rounded_down.empty else 0

        print("wowza\n",min_value)
        # epic_gamer_result = extra_crafts_based_on_ingredients.apply(smallest_non_zero_round_down)
        # print("part 2 extra_crafts_based_on_ingredients:\n", min_value)
        # extra_crafts_based_on_ingredients = ingredients_returned // merged_data['Quantity'] # amount of crafts can be done
        # print("extra_crafts_based_on_ingredients\n",extra_crafts_based_on_ingredients)
        # Calculate possible extra crafts based on remaining focus
        extra_crafts_based_on_focus = remaining_focus // focus
        print("working")
        print("extra_crafts_based_on_ingredients",extra_crafts_based_on_ingredients)
        print("extra_crafts_based_on_focus", extra_crafts_based_on_focus)
        # Determine the actual number of extra crafts possible
        actual_extra_crafts = min(extra_crafts_based_on_ingredients, extra_crafts_based_on_focus)

        if actual_extra_crafts > 0:
            total_crafts += actual_extra_crafts
            remaining_focus -= actual_extra_crafts * focus
            remaining_ingredients = ingredients_returned  # Update remaining ingredients for next iteration
        else:
            break

    return total_crafts


In [176]:
def calculate_with_reuse(recipes_df, ingredients_df, focus_df, cities, budget, focus_percentage, focus_limit):
    results = {}

    for recipe in recipes_df.columns[1:]:  # Exclude the 'Item' column
        results[recipe] = {}
        recipe_data = recipes_df[['Item', recipe]].dropna()
        recipe_data.columns = ['Item', 'Quantity']
        
        # Assuming focus_df has a column for each recipe with a single focus cost value per recipe
        focus_cost_per_batch = focus_df[recipe].iloc[0]  # Access the focus cost for the singlular recipe
        
        for city in cities:
            buy_column = f'{city} Buy Order'
            sell_column = f'{city} Sell Order'
            prices = ingredients_df[['Item', buy_column, sell_column]]
            merged_data = recipe_data.merge(prices, on='Item', how='left')

            merged_data['Cost Per Unit'] = merged_data['Quantity'] * merged_data[buy_column]
            single_recipe_cost = merged_data['Cost Per Unit'].sum() # gets the total cost per unit for a single recipe
            # check if a recipe cost is 0 if it is 0 then everything else is 0
            achievable_items, limiting_factor = single_item_achievable(single_recipe_cost, focus_cost_per_batch, focus_limit, budget)
            initial_batches = achievable_items
            if achievable_items != 0 and achievable_items is not None:
                    achievable_items = craft_until_focus_depleted(achievable_items, focus_cost_per_batch, focus_limit, focus_percentage, merged_data)
            total_batches = achievable_items
            total_focus_cost = achievable_items * focus_cost_per_batch

            #if single_recipe_cost > 0:
                # initial_batches = budget // single_recipe_cost
                # if initial_batches * focus_cost_per_batch
                #initial_batches = budget // single_recipe_cost
                
                #total_batches = initial_batches
                #ingredients_remaining = initial_batches * merged_data['Quantity']
                # ingredients_remaining = achievable_items * merged_data['Quantity']
                # total_focus_cost = 0

                # while True:
                #     ingredients_returned = ingredients_remaining * (focus_percentage / 100)
                #     additional_batches = (ingredients_returned // merged_data['Quantity']).min()
                #     if additional_batches < 1:
                #         break

                #     # Calculate the focus cost for these additional batches
                #     additional_focus_cost = additional_batches * focus_cost_per_batch
                #     if total_focus_cost + additional_focus_cost > focus_limit:  # Check if adding this cost exceeds the limit
                #         break

                #     total_batches += additional_batches
                #     total_focus_cost += additional_focus_cost
                #     ingredients_remaining = additional_batches * merged_data['Quantity']

            #else:
                #total_batches = 0


            # Calculate revenue and net profit
            revenue = total_batches * merged_data[sell_column].iloc[0]  # Assume uniform sell price
            profit = revenue - (single_recipe_cost * initial_batches) #- total_focus_cost

            results[recipe][city] = {
                'Initial Cost': single_recipe_cost * initial_batches,
                'Total Batches': total_batches,
                'Total Focus Cost': total_focus_cost,
                'Revenue': revenue,
                'Profit': profit
            }

    return results

# Example usage:
budget = int(input("Enter your budget: "))
focus_percentage = float(input("Enter the focus percentage (0-100): "))
focus_limit = int(input("Enter the focus limit: "))
results = calculate_with_reuse(recipes_df, ingredients_df, focus_df, cities, budget, focus_percentage, focus_limit)

# Display the results
for recipe, city_data in results.items():
    print(f"\n{recipe}:")
    for city, values in city_data.items():
        print(f"{city}:")
        print(f"  Initial Cost: {values['Initial Cost']}")
        print(f"  Total Batches: {values['Total Batches']}")
        print(f"  Total Focus Cost: {values['Total Focus Cost']}")
        print(f"  Revenue: {values['Revenue']}")
        print(f"  Profit: {values['Profit']}")


ingredients_returned
 0       0.0
1     315.2
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15    630.4
16      0.0
17      0.0
18      0.0
19      0.0
20    315.2
Name: Quantity, dtype: float64
part 1 extra_crafts_based_on_ingredients:
 0      0.0
1     78.8
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15    78.8
16     0.0
17     0.0
18     0.0
19     0.0
20    78.8
Name: Quantity, dtype: float64


KeyError: 'Quantity'

In [ ]:
# for city in cities:
#     # Extract sell and buy order columns for the current city
#     sell_column = f'{city} Sell Order'
#     buy_column = f'{city} Buy Order'

#     # Merge sell prices with recipes
#     current_sell_prices = ingredients_df[['Item', sell_column]]
#     combined_sell = recipes_long.merge(current_sell_prices, on='Item', how='left')
#     combined_sell['Ingredient_Cost'] = combined_sell['Quantity'] * combined_sell[sell_column]
    
#     # Aggregate to get total cost per recipe
#     sell_costs = combined_sell.groupby('Recipe')['Ingredient_Cost'].sum().rename(f'Sell Order{city}')
    
#     # Merge buy prices with recipes
#     current_buy_prices = ingredients_df[['Item', buy_column]]
#     combined_buy = recipes_long.merge(current_buy_prices, on='Item', how='left')
#     combined_buy['Ingredient_Cost'] = combined_buy['Quantity'] * combined_buy[buy_column]
    
#     # Aggregate to get total cost per recipe
#     buy_costs = combined_buy.groupby('Recipe')['Ingredient_Cost'].sum().rename(f'Buy Order{city}')

#     # Store in the dictionary
#     total_costs[f'{city}'] = pd.concat([sell_costs, buy_costs], axis=1)

# # Combine all data into a single DataFrame for display
# all_costs_df = pd.concat(total_costs, axis=1)

# # Display the total cost per recipe for sell and buy orders for each city, sorted as requested
# print(all_costs_df.head()[:5])

In [ ]:
# all_costs_df.to_csv(file_path)
# print(f'File saved as: {file_path}')

In [ ]:
print(results_df.info(memory_usage='deep'))


NameError: name 'results_df' is not defined

In [ ]:
combined_df.to_csv(file_path)
print(f'File saved as: {file_path}')
